In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Verilerin küçük bir kısmı deneme için.
data = pd.read_csv('shortpdata.csv')

# ilk 5 satır
print(data.head())

print(data.describe())

sns.countplot(data['type'])
plt.title('İşlem Türlerinin Dağılımı')
plt.show()

# Dolandırıcılık durumunun dağılımı
sns.countplot(data['isFraud'])
plt.title('Dolandırıcılık Dağılımı')
plt.show()

# Eski ve yeni bakiye ile dolandırıcılık arasındaki ilişki
sns.scatterplot(x='oldbalanceOrg', y='newbalanceOrig', hue='isFraud', data=data)
plt.title('Bakiyeler ve Dolandırıcılık İlişkisi')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Kategorik veriler (type) için one-hot encoding
encoder = OneHotEncoder()
encoded_types = encoder.fit_transform(data[['type']]).toarray()

# Encoding yapılmış veriyi veri setine ekleyelim
encoded_df = pd.DataFrame(encoded_types, columns=encoder.get_feature_names_out(['type']))
data = pd.concat([data, encoded_df], axis=1)

# Gereksiz sütunları düş (örneğin nameOrig ve nameDest gibi)
data = data.drop(['type', 'nameOrig', 'nameDest'], axis=1)

# Özellikleri (X) ve hedef değişkeni (y) belirle
X = data.drop(['isFraud'], axis=1)
y = data['isFraud']

# Veriyi eğitim ve test kümelerine ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Lojistik Regresyon modelini oluştur ve eğit
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Test verisi üzerinde tahmin yap
y_pred = model.predict(X_test)

# Modelin performansını değerlendirme
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Ridge Regression modelini oluştur
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)

# Test verisi üzerinde tahmin yap
y_pred_ridge = ridge_model.predict(X_test)

# Modelin performansını değerlendirme
mse = mean_squared_error(y_test, y_pred_ridge)
mae = mean_absolute_error(y_test, y_pred_ridge)

print(f"Ridge Regression MSE: {mse}")
print(f"Ridge Regression MAE: {mae}")


In [ ]:
#Unsupervised Learning
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Verilerin ölçeklendirilmesi
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# KMeans modelini oluştur ve eğit
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X_scaled)

# Tahmin edilen kümeler
clusters = kmeans.predict(X_scaled)

# Sonuçları veri setine ekleyelim
data['Cluster'] = clusters

# Küme sonuçlarının analiz edilmesi
print(data[['isFraud', 'Cluster']].groupby('Cluster').mean())

# Kümeleme sonuçlarını görselleştirelim
sns.scatterplot(x='oldbalanceOrg', y='newbalanceOrig', hue='Cluster', data=data)
plt.title('KMeans Kümeleme Sonuçları')
plt.show()
